In [397]:
# Standard stack
import datetime
import pandas as pd
import numpy as np
import re

# Visualization
from pandas_profiling import ProfileReport
#import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
import tensorflow as tf
import tensorflow_decision_forests as tfdf
try:
    from wurlitzer import sys_pipes
except:
    from colabtools.googlelog import CaptureLog as sys_pipes

# Scikit-learn packages
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# display
from IPython.core.magic import register_line_magic
from IPython.display import Javascript

In [398]:
KAGGLE_EVAL_METRIC = 'logloss' # string name for loss function in xgboost

TRAIN_PATH = "data/train.csv"
TEST_PATH = "data/test.csv"

label = "rain_tomorrow"

# Load Data

In [399]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [400]:
train.head(1)

,id,date,location,min_temp,max_temp,rainfall,evaporation,sunshine,wind_gust_dir,wind_gust_speed,...,humidity9am,humidity3pm,pressure9am,pressure3pm,cloud9am,cloud3pm,temp9am,temp3pm,rain_today,rain_tomorrow
0,6364,2010-11-11,BadgerysCreek,16.2,28.9,11.4,NaN,NaN,SW,37.0,...,78.0,52.0,1017.2,1011.7,NaN,NaN,20.3,27.9,1.0,0


In [401]:
16.2 - 27

-10.8

In [402]:
# Inputation
# Replaces numerical NaN (representing missing values in Pandas Dataframe) with 0s.
# ...Neural Nets don't work well with numerical NaNs.
def impute_numbers(dataset_df):
    for col in dataset_df.columns:
        if dataset_df[col].dtype not in [str, object]:
            dataset_df[col] = dataset_df[col].fillna(0)
    return dataset_df

# Split the dataset into a training and testing dataset.
def split_dataset(dataset, test_ratio=0.20):
    """Splits a panda dataframe in two."""
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

def prepare_dataset(dataset, train=True):
    dataset['humidity_diff'] = dataset['humidity3pm'] - dataset['humidity9am']
    dataset['3pm_temp_change'] = dataset['min_temp'] - dataset['temp3pm']
    dataset['3pm_temp_change'] = dataset['3pm_temp_change'].abs()
    dataset['3pm_max_change'] = dataset['max_temp'] - dataset['temp3pm']
    dataset['3pm_max_change'] = dataset['3pm_max_change'].abs()
    dataset['temp_change'] = dataset['min_temp'] - dataset['max_temp']
    if train == True:
        features = ['rain_tomorrow','humidity9am', 'humidity3pm', 'cloud9am', 
            'cloud3pm', 'wind_gust_speed','temp3pm','rain_today','3pm_temp_change','3pm_max_change','temp_change','humidity_diff']
    else:
        features = ['humidity9am', 'humidity3pm', 'cloud9am', 
            'cloud3pm', 'wind_gust_speed','temp3pm','rain_today',
                   '3pm_temp_change','3pm_max_change','temp_change','humidity_diff']
    dataset = dataset[features]
    dataset = impute_numbers(dataset)
    return dataset


In [403]:
submit = test
train = prepare_dataset(train)
test = prepare_dataset(test, False)


<ipython-input-402-ea705d413165>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df[col] = dataset_df[col].fillna(0)


In [404]:
train_ds_pd, test_ds_pd = split_dataset(train)
print("{} examples in training, {} examples for testing.".format(len(train_ds_pd), len(test_ds_pd)))

27328 examples in training, 6863 examples for testing.


In [405]:
train_ds_pd

,rain_tomorrow,humidity9am,humidity3pm,cloud9am,cloud3pm,wind_gust_speed,temp3pm,rain_today,3pm_temp_change,3pm_max_change,temp_change,humidity_diff
0,0,78.0,52.0,0.0,0.0,37.0,27.9,1.0,11.7,1.0,-12.7,-26.0
1,1,81.0,65.0,7.0,7.0,35.0,24.4,0.0,9.8,3.9,-13.7,-16.0
3,0,66.0,37.0,0.0,0.0,35.0,24.7,0.0,10.7,1.2,-11.9,-29.0
5,0,72.0,53.0,5.0,3.0,35.0,16.2,1.0,7.7,0.1,-7.8,-19.0
6,0,57.0,46.0,0.0,0.0,33.0,29.9,0.0,13.7,1.8,-15.5,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
34185,0,74.0,65.0,7.0,7.0,50.0,24.5,1.0,8.0,1.7,-9.7,-9.0
34186,1,81.0,86.0,8.0,8.0,15.0,23.7,1.0,3.3,3.5,-6.8,5.0
34187,1,88.0,51.0,7.0,5.0,76.0,12.5,1.0,6.6,0.9,-7.5,-37.0
34189,1,85.0,88.0,7.0,7.0,28.0,6.2,1.0,1.0,2.2,-3.2,3.0


In [406]:
# Convert the datasets into tensorflow datasets
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)
score_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test)

# Feature Engineer

## Define Features and Target
Define categorical and numeric features manually because sometimes cols of type float/int should be categorical and vice versa

In [407]:
train_ds_pd[features].head(1)

,humidity9am,humidity3pm,cloud9am,cloud3pm,wind_gust_speed,temp3pm,rain_today
0,78.0,52.0,0.0,0.0,37.0,27.9,1.0


In [408]:
input_1 = tf.keras.Input(shape=(1,), name="humidity9am", dtype="float")
input_2 = tf.keras.Input(shape=(1,), name="humidity3pm", dtype="float")
input_3 = tf.keras.Input(shape=(1,), name="cloud9am", dtype="float")
input_4 = tf.keras.Input(shape=(1,), name="cloud3pm", dtype="float")
input_5 = tf.keras.Input(shape=(1,), name="wind_gust_speed", dtype="float")
input_6 = tf.keras.Input(shape=(1,), name="temp3pm", dtype="float")
input_7 = tf.keras.Input(shape=(1,), name="rain_today", dtype="float")
#input_8 = tf.keras.Input(shape=(1,), name="location", dtype="string")

nn_raw_inputs = [input_1, input_2,input_3,input_4,input_5,input_6,input_7]

In [409]:
Normalization = tf.keras.layers.experimental.preprocessing.Normalization
CategoryEncoding = tf.keras.layers.experimental.preprocessing.CategoryEncoding
StringLookup = tf.keras.layers.experimental.preprocessing.StringLookup

# Numerical
values = train_ds_pd["humidity9am"].values
input_1_normalizer = Normalization()
input_1_normalizer.adapt(values)

values = train_ds_pd["humidity3pm"].values
input_2_normalizer = Normalization()
input_2_normalizer.adapt(values)

values = train_ds_pd["cloud9am"].values
input_3_normalizer = Normalization()
input_3_normalizer.adapt(values)

values = train_ds_pd["cloud3pm"].values
input_4_normalizer = Normalization()
input_4_normalizer.adapt(values)

values = train_ds_pd["wind_gust_speed"].values
input_5_normalizer = Normalization()
input_5_normalizer.adapt(values)

values = train_ds_pd["temp3pm"].values
input_6_normalizer = Normalization()
input_6_normalizer.adapt(values)

values = train_ds_pd["rain_today"].values
input_7_normalizer = Normalization()
input_7_normalizer.adapt(values)

#values = train_ds_pd["location"].values
#input_8_indexer = StringLookup(max_tokens=100000)
#input_8_indexer.adapt(values)
#input_8_onehot = CategoryEncoding(output_mode="binary", max_tokens=100000)


normalized_input_1 = input_1_normalizer(input_1)
normalized_input_2 = input_2_normalizer(input_2)
normalized_input_3 = input_3_normalizer(input_3)
normalized_input_4 = input_4_normalizer(input_4)
normalized_input_5 = input_5_normalizer(input_5)
normalized_input_6 = input_6_normalizer(input_6)
normalized_input_7 = input_7_normalizer(input_7)
#normalized_input_8 = input_8_onehot(input_8_indexer(input_8))


nn_processed_inputs = [normalized_input_2,
                      normalized_input_4,normalized_input_5,normalized_input_6,
                      normalized_input_7]

In [410]:
y = tf.keras.layers.Concatenate()(nn_processed_inputs)
y = tf.keras.layers.Dense(16, activation=tf.nn.relu6)(y)
last_layer = tf.keras.layers.Dense(8, activation=tf.nn.relu, name="last")(y)

# "3" for the three label classes. If it were a binary classification, the
# output dim would be 1.
classification_output = tf.keras.layers.Dense(2)(y)

nn_model = tf.keras.models.Model(nn_raw_inputs, classification_output)

# To reduce the risk of mistakes, group both the decision forest and the
# neural network in a single keras model.
nn_without_head = tf.keras.models.Model(inputs=nn_model.inputs, outputs=last_layer)
df_and_nn_model = tfdf.keras.GradientBoostedTreesModel(
    num_trees=500,
    growing_strategy="BEST_FIRST_GLOBAL",
    hyperparameter_template="better_default@v1",
    max_depth=8,
    split_axis="SPARSE_OBLIQUE",
    preprocessing=nn_without_head)

In [411]:
nn_model.inputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'humidity9am')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'humidity3pm')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'cloud9am')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'cloud3pm')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'wind_gust_speed')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'temp3pm')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'rain_today')>]

In [412]:
nn_model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=["accuracy"])

nn_model.fit(x=train_ds, validation_data=test_ds, epochs=10, verbose=False)
print("Evaluation:", nn_model.evaluate(test_ds))

/home/neil/anaconda3/envs/ds_py_3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:588: UserWarning: Input dict contained keys ['3pm_temp_change', '3pm_max_change', 'temp_change', 'humidity_diff'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


108/108 [==============================] - 0s 932us/step - loss: 0.3795 - accuracy: 0.8383
Evaluation: [0.3795132637023926, 0.8382631540298462]


In [413]:
df_and_nn_model.compile(metrics=["accuracy"])
df_and_nn_model.fit(x=train_ds)
print("Evaluation:", df_and_nn_model.evaluate(test_ds))

108/108 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8380
Evaluation: [0.0, 0.8379717469215393]


In [414]:
# Submit 

In [428]:
scores = df_and_nn_model.predict(score_ds)

In [429]:
scores

array([[0.06492948],
       [0.5113515 ],
       [0.07484271],
       ...,
       [0.8413882 ],
       [0.04349693],
       [0.03286444]], dtype=float32)

In [430]:
submit['rain_tomorrow'] = scores[:,0]

In [431]:
submission

,id,rain_tomorrow
0,18416,0.349131
1,32197,-1.830140
2,7167,0.631383
3,2036,0.724348
4,34361,-0.941905
...,...,...
14648,42748,-0.407030
14649,699,0.167941
14650,7018,-1.761055
14651,41137,0.871674


In [422]:
submission = submit[['id','rain_tomorrow']]

In [423]:
submission.to_csv('tf-submission.csv', index=False)

In [424]:
! kaggle competitions submit -c sliced-s01e04-knyna9 -f tf-submission.csv -m "no date, no feature engineer"

100%|█████████████████████████████████████████| 237k/237k [00:00<00:00, 379kB/s]
Successfully submitted to SLICED s01e04